In [ ]:
import collections
from ortools.sat.python import cp_model
import numpy as np

# Environment Initialization

In [ ]:
def initialize(instance_path):
    """
        populate job_operation_map using the instance file input
        :param instance_path: a string representing file path
    """
    # input instance description for environment initialization
    instance_path = instance_path
    try:
        file_handle = open(instance_path, 'r')
    except OSError:
        print("Could not open/read file:", instance_path)

    lines_list = file_handle.readlines()
    job_operation_map = []
    # first line consists of # of jobs in total, # of machines in total
    job_total, machine_total = [int(x) for x in lines_list[0].split()]
    # read through each job description
    for job_index in range(len(lines_list) - 1):
        job_description = np.array([int(val) for val in lines_list[job_index + 1].split()])
        job_operation_map.append([])
        # populate job_description_map
        job_operation_map[job_index] = populate_job_description_map(job_description, job_index)
    file_handle.close()
    return job_operation_map, job_total, machine_total

def populate_job_description_map(job_description, job_index):
    """
        populate the corresponding fields in job_description_map
        :param job_description: a string representing job description
        :param job_index: an integer representing current job index being populated
    """
    job_array = []
    # read job description from left to right
    description_pivot = 1
    operation_index = 0
    # operation_index
    while description_pivot < len(job_description):
        operation_array = []
        # operation_description length = 2 * # of machines capable of executing current operation at the pivot
        operation_description_end = 2 * job_description[description_pivot] + description_pivot
        # read the current description of operation
        description_pivot += 1
        while description_pivot <= operation_description_end:
            # Following the format of operation description:
            # machine index , time it takes for the current operation
            machine_index = job_description[description_pivot]
            operation_duration = job_description[description_pivot + 1]
            operation_array.append((operation_duration, machine_index))
            description_pivot += 2
        operation_index += 1
        job_array.append(operation_array)
    return job_array

In [ ]:
env_s, job_total_s, machine_total_s = initialize("../Instances/instance1.txt")
env_m, job_total_m, machine_total_m = initialize("../Instances/instance5.txt")
env_l, job_total_l, machine_total_l = initialize("../Instances/instance4.txt")
env_xl, job_total_xl, machine_total_xl = initialize("../Instances/instance3.txt")

In [ ]:
env_names = ["Small", "Medium", "Large", "Extra Large"]
env_list = [env_s, env_m, env_l, env_xl]
job_total_list = [job_total_s, job_total_m, job_total_l, job_total_xl]
machine_total_list = [machine_total_s, machine_total_m, machine_total_l, machine_total_xl]

# Algorithm

In [ ]:

class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__solution_count = 0

    def on_solution_callback(self):
        """Called at each new solution."""
        # print('Solution %i, time = %f s, objective = %i' %
        #       (self.__solution_count, self.WallTime(), self.ObjectiveValue()))
        self.__solution_count += 1

In [ ]:
def flexible_jobshop(job_operation_map, job_total, machine_total, print_solution = False):
    """Solve a small flexible jobshop problem."""
    # Data part.
    jobs = job_operation_map

    num_jobs = job_total
    all_jobs = range(num_jobs)

    num_machines = machine_total
    all_machines = range(num_machines)

    # Model the flexible jobshop problem.
    model = cp_model.CpModel()

    horizon = 0
    for job in jobs:
        for task in job:
            max_task_duration = 0
            for alternative in task:
                max_task_duration = max(max_task_duration, alternative[0])
            horizon += max_task_duration

    # print('Horizon = %i' % horizon)

    # Global storage of variables.
    intervals_per_resources = collections.defaultdict(list)
    starts = {}  # indexed by (job_id, task_id).
    presences = {}  # indexed by (job_id, task_id, alt_id).
    job_ends = []

    # Scan the jobs and create the relevant variables and intervals.
    for job_id in all_jobs:
        job = jobs[job_id]
        num_tasks = len(job)
        previous_end = None
        for task_id in range(num_tasks):
            task = job[task_id]

            min_duration = task[0][0]
            max_duration = task[0][0]

            num_alternatives = len(task)
            all_alternatives = range(num_alternatives)

            for alt_id in range(1, num_alternatives):
                alt_duration = task[alt_id][0]
                min_duration = min(min_duration, alt_duration)
                max_duration = max(max_duration, alt_duration)

            # Create main interval for the task.
            suffix_name = '_j%i_t%i' % (job_id, task_id)
            start = model.NewIntVar(0, horizon, 'start' + suffix_name)
            duration = model.NewIntVar(min_duration, max_duration,
                                       'duration' + suffix_name)
            end = model.NewIntVar(0, horizon, 'end' + suffix_name)
            interval = model.NewIntervalVar(start, duration, end,
                                            'interval' + suffix_name)

            # Store the start for the solution.
            starts[(job_id, task_id)] = start

            # Add precedence with previous task in the same job.
            if previous_end is not None:
                model.Add(start >= previous_end)
            previous_end = end

            # Create alternative intervals.
            if num_alternatives > 1:
                l_presences = []
                for alt_id in all_alternatives:
                    alt_suffix = '_j%i_t%i_a%i' % (job_id, task_id, alt_id)
                    l_presence = model.NewBoolVar('presence' + alt_suffix)
                    l_start = model.NewIntVar(0, horizon, 'start' + alt_suffix)
                    l_duration = task[alt_id][0]
                    l_end = model.NewIntVar(0, horizon, 'end' + alt_suffix)
                    l_interval = model.NewOptionalIntervalVar(
                        l_start, l_duration, l_end, l_presence,
                        'interval' + alt_suffix)
                    l_presences.append(l_presence)

                    # Link the primary/global variables with the local ones.
                    model.Add(start == l_start).OnlyEnforceIf(l_presence)
                    model.Add(duration == l_duration).OnlyEnforceIf(l_presence)
                    model.Add(end == l_end).OnlyEnforceIf(l_presence)

                    # Add the local interval to the right machine.
                    intervals_per_resources[task[alt_id][1]].append(l_interval)

                    # Store the presences for the solution.
                    presences[(job_id, task_id, alt_id)] = l_presence

                # Select exactly one presence variable.
                model.AddExactlyOne(l_presences)
            else:
                intervals_per_resources[task[0][1]].append(interval)
                presences[(job_id, task_id, 0)] = model.NewConstant(1)

        job_ends.append(previous_end)

    # Create machines constraints.
    for machine_id in all_machines:
        intervals = intervals_per_resources[machine_id]
        if len(intervals) > 1:
            model.AddNoOverlap(intervals)

    # Makespan objective
    makespan = model.NewIntVar(0, horizon, 'makespan')
    model.AddMaxEquality(makespan, job_ends)
    model.Minimize(makespan)

    # Solve model.
    solver = cp_model.CpSolver()
    solution_printer = SolutionPrinter()
    status = solver.Solve(model, solution_printer)

    if print_solution:
        # Print final solution.
        for job_id in all_jobs:
            print('Job %i:' % job_id)
            for task_id in range(len(jobs[job_id])):
                start_value = solver.Value(starts[(job_id, task_id)])
                machine = -1
                duration = -1
                selected = -1
                for alt_id in range(len(jobs[job_id][task_id])):
                    if solver.Value(presences[(job_id, task_id, alt_id)]):
                        duration = jobs[job_id][task_id][alt_id][0]
                        machine = jobs[job_id][task_id][alt_id][1]
                        selected = alt_id
                print(
                    '  task_%i_%i starts at %i (alt %i, machine %i, duration %i)' %
                    (job_id, task_id, start_value, selected, machine, duration))

    # print('Solve status: %s' % solver.StatusName(status))
    print('Optimal makespan: %i' % solver.ObjectiveValue())
    # print('Statistics')
    # print('  - conflicts : %i' % solver.NumConflicts())
    # print('  - branches  : %i' % solver.NumBranches())
    # print('  - wall time : %f s' % solver.WallTime())
    return solver.ObjectiveValue()

# Analysis

In [68]:
solutions_rs = [0]*4
for i in range(4):
    print('Solution found by OR-Tools for {} environment'.format(env_names[i]))
    solutions_rs[i] = flexible_jobshop(env_list[i], job_total_list[i], machine_total_list[i])

Solution found by OR-Tools for Small environment
Optimal makespan: 53
Solution found by OR-Tools for Medium environment
Optimal makespan: 55
Solution found by OR-Tools for Large environment
Optimal makespan: 40
Solution found by OR-Tools for Extra Large environment
Optimal makespan: 927
